## Deploy Functions Modelo

In [7]:
import os

# Definir o diretório que você quer listar os arquivos
directory = 'trained_v1/'

# Listar todos os itens no diretório e filtrar apenas os arquivos
models = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# Printar os nomes dos arquivos
print(models)


['logistic_regression_model_v1_bl_target_10_15d.joblib', 'logistic_regression_model_v1_bl_target_10_30d.joblib', 'logistic_regression_model_v1_bl_target_10_7d.joblib', 'logistic_regression_model_v1_bl_target_15_15d.joblib', 'logistic_regression_model_v1_bl_target_15_30d.joblib', 'logistic_regression_model_v1_bl_target_15_7d.joblib', 'logistic_regression_model_v1_bl_target_20_15d.joblib', 'logistic_regression_model_v1_bl_target_20_30d.joblib', 'logistic_regression_model_v1_bl_target_20_7d.joblib', 'logistic_regression_model_v1_bl_target_25_15d.joblib', 'logistic_regression_model_v1_bl_target_25_30d.joblib', 'logistic_regression_model_v1_bl_target_25_7d.joblib']


In [ ]:
# Aqui entra a regra de seleção para o modelos
# for model in models:
    
import joblib

# clf = joblib.load(r'D:/Github/Forked/crypto_technical_analysis/models/logistic_regression_model_bl_target_10_7d.joblib')
# clf = joblib.load(r'D:/Github/Forked/crypto_technical_analysis/models/trained_withoutSymbols/logistic_regression_model_without_symbolbl_target_10_15d.joblib')
clf = joblib.load(r'D:/Github/Forked/crypto_technical_analysis/models/trained_v1/logistic_regression_model_v1_bl_target_10_7d.joblib')

In [9]:
from pathlib import Path
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

input_path = r'D:\Github\Forked\crypto_technical_analysis\files\crypto_data_with_indicators.parquet'

dados = pd.read_parquet(input_path)

In [10]:
# Constants
target_list_bol =   [
    # boleanos
    'bl_target_10_7d','bl_target_15_7d','bl_target_20_7d','bl_target_25_7d',
    'bl_target_10_15d','bl_target_15_15d','bl_target_20_15d','bl_target_25_15d', 
    'bl_target_10_30d','bl_target_15_30d','bl_target_20_30d','bl_target_25_30d' 
]

target_list_val =   [
    # percentual
    'target_10_7d','target_15_7d','target_20_7d','target_25_7d',
    'target_10_15d','target_15_15d','target_20_15d','target_25_15d', 
    'target_10_30d','target_15_30d','target_20_30d','target_25_30d', 
]

remove_target_list = target_list_bol + target_list_val

removing_cols = ['Date', 'Symbol', 'Dividends', 'Stock Splits']

In [45]:
def eval_data(dados, date_eval = None):

    dados['Date'] = pd.to_datetime(dados['Date'])
    dados['Date'] = dados['Date'].dt.strftime('%Y-%m-%d')

    if date_eval is None or date_eval == '':
        choosen_date = dados['Date'].max()
    else:
        choosen_date = date_eval
        
    filtered_data = dados[dados['Date'] == choosen_date]

    return filtered_data

In [46]:
def build_dummies(filtered_data, target_list, remove_cols):
    
    dados_x = filtered_data.drop(filtered_data[target_list], axis=1)
    dados_x = dados_x.drop(dados_x[remove_cols], axis=1)

    dummies_build = pd.get_dummies(dados_x)

    dummies_prep = dummies_build.dropna()

    return dummies_prep

In [47]:
dataset_ref = eval_data(dados, '')

In [48]:
dummies_input = build_dummies(dataset_ref, remove_target_list, removing_cols)

In [37]:
def data_clean(dados:pd.DataFrame, target_list:list, data_return:str):
    # Removing NA
    dados_treat = dados.dropna()

    # Substituindo valores infinitos por NaN
    dados_treat.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Removendo linhas com valores NaN
    dados_treat.dropna(inplace=True)

    # Removing cols that won't be used in the model
    # removing_cols = ['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
    # removing_cols = ['Date']
    # removing_cols = ['Date', 'Symbol']
    removing_cols = ['Date', 'Symbol', 'Dividends', 'Stock Splits']

    # Define the target in a list of target (for futher iteration)
    dados_y = dados_treat[target_list]

    # Removing target from base to avoid data leakage
    dados_x = dados_treat.drop(dados_treat[target_list], axis=1)
    dados_x = dados_x.drop(dados_x[removing_cols], axis=1)

    if data_return == 'Y':
        return dados_y
    else:
        return dados_x

In [34]:
dados_x_all = data_clean(dados, remove_target_list, 'X')
dados_x_all_dummies = pd.get_dummies(dados_x_all)

C:\Users\mathe\AppData\Local\Temp\ipykernel_2420\761462767.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_treat.replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\mathe\AppData\Local\Temp\ipykernel_2420\761462767.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_treat.dropna(inplace=True)


In [51]:
def padronize_dummies(dummies_input, dummies_ref):
    # Encontrar colunas que estão no treinamento mas não na validação
    missing_cols = set(dummies_ref.columns) - set(dummies_input.columns)

    # # Adicionar as colunas faltantes no conjunto de validação, preenchidas com zeros
    for col in missing_cols:
        dummies_input[col] = False

    # Reordenar as colunas no conjunto de validação para corresponder ao conjunto de treinamento
    valid_dummies = dummies_input[dummies_ref.columns]

    return valid_dummies

In [52]:
padronized_dummies = padronize_dummies(dummies_input, dados_x_all_dummies)

In [43]:
x = dados[[]]
x

""
0
1
2
3
4
...
892083
892084
892085
892086


In [44]:
def add_proba_target(classifier, dummies_input, dataset_ref, col_name_output):

    #Fazendo a previsão das probabilidades
    proba = classifier.predict_proba(dummies_input)

    # Probabilidade de ser o target:
    proba_target = proba[:,1] # array

    proba_dataset = dummies_input[[]] # pegando apenas os índices do dataset de input (que já contém os dados de retorno)

    proba_dataset[col_name_output] = proba_target

    # proba_crypto_date = dataset_ref[['Symbol', 'Date', 'Close']]

    build_dataset_proba = pd.merge(dataset_ref, proba_dataset, left_index=True, right_index=True)
    
    return build_dataset_proba

In [14]:
def build_var_name(model_name, prefix):

    string_name = model_name.split('.')[0]

    splited_list = string_name.split('_')
    
    build_name = splited_list[0][0] + splited_list[1][0]  + prefix + splited_list[-2] + '_' + splited_list[-1]
    
    return build_name


In [15]:
build_var_name('logistic_regression_model_v1_bl_target_10_30d.joblib', '_pb_')

'lr_pb_10_30d'

In [53]:
compiled_dataset = dataset_ref[['Symbol', 'Date', 'Close']]

add_proba_target(clf, padronized_dummies, compiled_dataset, 'target_pb')
# result.sort_values(by='proba_target', ascending=False)

c:\Users\mathe\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
C:\Users\mathe\AppData\Local\Temp\ipykernel_2420\3155700715.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proba_dataset[col_name_output] = proba_target


,Symbol,Date,Close,target_pb
205,1CAT-USD,2024-07-12,0.002353,0.0
1502,1INCH-USD,2024-07-12,0.416116,0.0
1916,AA-USD,2024-07-12,0.505551,0.0
3297,AAVE-USD,2024-07-12,99.630486,0.0
5399,ABBC-USD,2024-07-12,0.008688,0.0
...,...,...,...,...
880908,ZELIX-USD,2024-07-12,0.000090,0.0
883294,ZEN-USD,2024-07-12,10.808343,0.0
884653,ZETA-USD,2024-07-12,0.640986,0.0
888029,ZIL-USD,2024-07-12,0.016214,0.0


In [ ]:
### outras funções

In [ ]:
# from datetime import datetime
# from pathlib import Path
# import pyarrow.parquet as pq
# import pyarrow as pa

import json
import requests as re
import yfinance as yf
import pandas as pd


def get_kucoin_symbols():
    resp = re.get('https://api.kucoin.com/api/v2/symbols')
    ticker_list = json.loads(resp.content)
    symbols_list = [ticker['symbol'] for ticker in ticker_list['data'] if str(ticker['symbol'][-4:]) == 'USDT']
    return symbols_list

def fetch_crypto_data(symbols, start_date):
    df_list = []

    for count, symbol in enumerate(symbols):
        print(f'Processing {symbol} ({count + 1} of {len(symbols)})')

        # Get historical data using yfinance for the given symbol since start_date
        crypto_data = yf.Ticker(symbol).history(start=start_date)

        # Add a 'symbol' column to the DataFrame to identify the cryptocurrency
        crypto_data['Symbol'] = symbol

        # Append the DataFrame to the list if it's not empty
        if not crypto_data.empty:
            df_list.append(crypto_data)

    # Concatenate the list of DataFrames into a single DataFrame
    if df_list:
        return pd.concat(df_list)
    else:
        return None

kucoin_symbols = get_kucoin_symbols()

data_hoje = pd.Timestamp.today()

# Formatando a data no formato 'YYYY-MM-DD'
data_hoje_formatada = data_hoje.strftime('%Y-%m-%d')

crypto_symbols = [symbol[:-1] for symbol in kucoin_symbols]

crypto_today = fetch_crypto_data(crypto_symbols, data_hoje_formatada)

In [ ]:

from indicators_util import add_indicators

# Add indicators to the DataFrame
indicators_dataframe = add_indicators(crypto_today)